In [1]:
from psycopg2 import connect as cnct
from matplotlib.ticker import FormatStrFormatter
from statsmodels.regression import linear_model
from sklearn.metrics import mean_squared_error as mse

import copy
import statsmodels.api as sm
import os 
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import datetime

matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})


DB_SETTINGS = {
    'dbname'   : 'orderlogs', 
    'port'     :  5432, 
    'user'     : 'postgres', 
    'host'     : 'localhost',
    'password' : ''
}

ENGINE_SETTINGS = 'postgresql://postgres@localhost:5432/orderlogs'

with cnct(**DB_SETTINGS) as conn:
    cur = conn.cursor()
    cur.execute("SELECT table_name FROM information_schema.tables WHERE table_schema='public'")
    TABLE_NAMES = cur.fetchall()
    TABLE_NAMES = sorted(list(map(lambda x: x[0], TABLE_NAMES)))

TABLE_NAMES_ORDERLOGS = list(filter(lambda x: len(x) == 8, TABLE_NAMES))
TABLE_NAMES_CHAINS = list(filter(lambda x: len(x) > 8, TABLE_NAMES))

PATH = os.path.abspath('')

In [2]:
def time_convertion(time: int, date=20190603) -> datetime.datetime:
    s_time = str(time)
    s_date = str(date)
    
    hours = int(s_time[:2])
    minutes = int(s_time[2:4])
    seconds = int(s_time[4:6])
    microseconds = int(s_time[6:])
    
    year = int(s_date[:4])
    month = int(s_date[4:6])
    day = int(s_date[6:])
    
    time_datetime = datetime.datetime(year=year, month=month, day=day, 
                                      hour=hours, minute=minutes, second=seconds, 
                                      microsecond=microseconds)
    
    return time_datetime

# 1. Индексы МосБиржи и РТС за май-июнь 2019

In [3]:
data_rts = pd.read_csv(os.path.join(PATH, 'additional_data_for_figures_tables/rts.csv'), sep=';')
data_rts = data_rts[['TRADEDATE', 'CLOSE']]

data_moex = pd.read_csv(os.path.join(PATH, 'additional_data_for_figures_tables/moex.csv'), sep=';')
data_moex = data_moex[['TRADEDATE', 'CLOSE']]

data_rts['TRADEDATE'] = list(map(lambda one: datetime.datetime.strptime(one, '%Y-%m-%d'), 
                                 data_rts['TRADEDATE']))
data_moex['TRADEDATE'] = list(map(lambda one: datetime.datetime.strptime(one, '%Y-%m-%d'), 
                                 data_moex['TRADEDATE']))

In [4]:
fig, ax1 = plt.subplots()

plt.xticks(rotation=45)
ax1.set_ylabel('RTS Value, USD', color='black')
line1, = ax1.plot(data_rts['TRADEDATE'], data_rts['CLOSE'], color='#C0C0C0')
    
ax2 = ax1.twinx()
ax2.set_ylabel('MOEX Value, RUB', color='black')
line2, = ax2.plot(data_moex['TRADEDATE'], data_moex['CLOSE'], color='#000000')

plt.legend((line1, line2), ('RTS', 'MOEX'))

plt.title('RTS, MOEX Dynamics, May-June, 2019')
fig.set_size_inches(w=5, h=4)
plt.savefig(os.path.join(PATH, 'figures_to_report/rts_moex.pgf'), bbox_inches='tight')

# 2. Активность торгов

In [5]:
data_sber = pd.read_csv(os.path.join(PATH, 
                                     'additional_data_for_figures_tables/activity_tables/activity_market_SBER.csv'))
data_sber.Date = list(map(lambda one: datetime.datetime.strptime(one, '%Y-%m-%d'), data_sber.Date))
data_sber.head()

,Date,Seccode,Number of Orders
0,2019-06-03,SBER,1957652
1,2019-06-04,SBER,1843734
2,2019-06-05,SBER,1358439
3,2019-06-06,SBER,1367139
4,2019-06-07,SBER,1077600


In [6]:
data_gazp = pd.read_csv(os.path.join(PATH, 
                                     'additional_data_for_figures_tables/activity_tables/activity_market_GAZP.csv'))
data_gazp.Date = list(map(lambda one: datetime.datetime.strptime(one, '%Y-%m-%d'), data_gazp.Date))
data_gazp.head()

,Date,Seccode,Number of Orders
0,2019-06-03,GAZP,4313098
1,2019-06-04,GAZP,2608790
2,2019-06-05,GAZP,1032804
3,2019-06-06,GAZP,1090416
4,2019-06-07,GAZP,708835


In [7]:
data_aptk = pd.read_csv(os.path.join(PATH, 
                                     'additional_data_for_figures_tables/activity_tables/activity_market_APTK.csv'))
data_aptk.Date = list(map(lambda one: datetime.datetime.strptime(one, '%Y-%m-%d'), data_aptk.Date))
data_aptk.head()

,Date,Seccode,Number of Orders
0,2019-06-03,APTK,12562
1,2019-06-04,APTK,11645
2,2019-06-05,APTK,15771
3,2019-06-06,APTK,19472
4,2019-06-07,APTK,7255


In [8]:
data_bane = pd.read_csv(os.path.join(PATH, 
                                     'additional_data_for_figures_tables/activity_tables/activity_market_BANE.csv'))
data_bane.Date = list(map(lambda one: datetime.datetime.strptime(one, '%Y-%m-%d'), data_bane.Date))
data_bane.head()

,Date,Seccode,Number of Orders
0,2019-06-03,BANE,11491
1,2019-06-04,BANE,8574
2,2019-06-05,BANE,5995
3,2019-06-06,BANE,8679
4,2019-06-07,BANE,7229


In [9]:
data_all = pd.read_csv(os.path.join(PATH, 
                                     'additional_data_for_figures_tables/activity_tables/activity_market_all.csv'))
data_all.Date = list(map(lambda one: datetime.datetime.strptime(one, '%Y-%m-%d'), data_all.Date))
data_all.head()

,Date,Number of Orders
0,2019-06-03,16642888
1,2019-06-04,13385835
2,2019-06-05,9761893
3,2019-06-06,9500628
4,2019-06-07,8301152


In [10]:
figure, (ax1, ax2) = plt.subplots(1, 2, sharey=False)

ax1.plot(data_sber.Date, data_sber['Number of Orders'], color='#000000')
ax1.ticklabel_format(axis='y', style='sci', scilimits=(0, 0))

ax2.plot(data_gazp.Date, data_gazp['Number of Orders'], color='#000000')
ax2.ticklabel_format(axis='y', style='sci', scilimits=(0, 0))

figure.autofmt_xdate()

ax1.title.set_text('SBER')
ax2.title.set_text('GAZP')

figure.set_size_inches(w=6, h=3)
plt.savefig(os.path.join(PATH, 'figures_to_report/activity_sber_gazp.pgf'), bbox_inches='tight')

In [11]:
figure, (ax1, ax2) = plt.subplots(1, 2, sharey=False)

ax1.plot(data_aptk.Date, data_aptk['Number of Orders'], color='#000000')
ax1.ticklabel_format(axis='y', style='sci', scilimits=(0, 0))

ax2.plot(data_bane.Date, data_bane['Number of Orders'], color='#000000')
ax2.ticklabel_format(axis='y', style='sci', scilimits=(0, 0))

figure.autofmt_xdate()

ax1.title.set_text('APTK')
ax2.title.set_text('BANE')

figure.set_size_inches(w=6, h=3)
plt.savefig(os.path.join(PATH, 'figures_to_report/activity_aptk_bane.pgf'), bbox_inches='tight')

In [12]:
figure, ax = plt.subplots(1, 1, sharey=True)

ax.plot(data_all.Date, data_all['Number of Orders'], color='#000000')
ax.ticklabel_format(axis='y', style='sci', scilimits=(0, 0))

figure.autofmt_xdate()

ax.title.set_text('All Market')

figure.set_size_inches(w=5, h=3)
plt.savefig(os.path.join(PATH, 'figures_to_report/activity_all.pgf'), bbox_inches='tight')

# 2. Книга лимитированных заявок

In [13]:
class LOB:
    '''This class replicates the logic for the limit order book building
    '''
    def __init__(self, ticker, date=None, time=None):
        self._ticker = ticker
        self._date = date
        self._time = time
    
    def table_build(self, date=None, time=None):
        '''This method builds the limit order book table on the specified date and time
        
        :date: the date in int format (for example, 20190603)
        :time: the time in int format (for example, 121059123456)
        :return: the pandas.DataFrame object with columns BUYSELL, ORDERNO, PRICE, VOLUME
        '''
        if date is None and time is None:
            date = self._date
            time = self._time
        elif date is None or time is None:
            raise ValueError('You must specify both date and time')
            
        with cnct(**DB_SETTINGS) as conn:
            cur = conn.cursor()

            table_name = date
            seccode = self._ticker

            query = '''\
            SELECT 
                tt."BUYSELL", tt."PRICE", sum("VOLUME") as "VOLUME"
            FROM
                (SELECT 
                    t."BUYSELL", t."ORDERNO", max(t."PRICE") as "PRICE", sum("VOLUME") as "VOLUME"
                FROM 
                    (SELECT
                        "BUYSELL", "ORDERNO", "ACTION", max("PRICE") as "PRICE",
                        CASE "ACTION" WHEN 1 THEN sum("VOLUME")
                                      WHEN 2 THEN sum(-"VOLUME")
                                      WHEN 0 THEN sum(-"VOLUME")
                        END as "VOLUME"
                    FROM 
                        "{}"
                    WHERE
                        "SECCODE" = '{}' AND "TIME" <= {} AND "PRICE" != 0
                    GROUP BY
                        "BUYSELL", 
                        "ORDERNO",
                        "ACTION") t
                GROUP BY
                    "BUYSELL",
                    "ORDERNO"
                HAVING
                    sum("VOLUME") > 0) tt
            GROUP BY
                "BUYSELL",
                "PRICE"
            ORDER BY
                "BUYSELL" DESC, 
                "PRICE" DESC;
            '''.format(table_name, seccode, time)
            query = " ".join(query.split())

            cur.execute(query)

            limit_order_book = pd.DataFrame(cur.fetchall(), 
                                            columns=['BUYSELL', 'PRICE', 'VOLUME'])
        return limit_order_book
    
    def lob_plot(self, date, time, buylevels=20, selllevels=20, path=None):
        '''This method builds the limit order book table and plot it on the specified date and time
        
        :date: the date in int format (for example, 20190603)
        :time: the time in int format (for example, 121059123456)
        :buylevels: the number of price levels of BID side, int
        :selllevels: the number of price levels of ASK side, int
        :path: whether you want to save the plot, set the path
        :return: None, matplotlib.pyplot hist returns
        '''
        
        lob = self.table_build(date, time)
        buy = (lob.loc[lob['BUYSELL'] == 'B', ['PRICE', 'VOLUME']]).reset_index(drop=True)
        sell = (lob.loc[lob['BUYSELL'] == 'S', ['PRICE', 'VOLUME']]).reset_index(drop=True)
        
        # code to make beautiful barplot
        bid = np.max(buy.PRICE)
        ask = np.min(sell.PRICE)
        plt.figure()
        plt.barh(sell.PRICE[-selllevels:].reset_index(drop=True), 
                 sell.VOLUME[-selllevels:].reset_index(drop=True), 
                 height=0.01, label = 'SELL', color = 'r')
        plt.barh(buy.PRICE[:buylevels].reset_index(drop=True), 
                 -buy.VOLUME[:buylevels].reset_index(drop=True),
                 height=0.01, label = 'BUY', color = 'g')
        plt.xticks(rotation=15)

        plt.xlabel('VOLUME')
        plt.ylabel('PRICE')
        plt.title('LOB for ' + self._ticker +\
                  ' at ' + str(date)[:4] + '-' + str(date)[4:6] + '-' + str(date)[6:] +\
                  ': ' +\
                  str(time)[:2] + ':' + str(time)[2:4] + ':' + str(time)[4:6] + '.' + str(time)[6:])
        plt.legend(['ASK = ' + str(ask), 
                    'BID = ' + str(bid)])
        plt.grid()
        
        if path is None:
            plt.show()
        else:
            plt.savefig(os.path.join(os.path.abspath(''), path), bbox_inches='tight')
            plt.close()

In [14]:
ticker_to_plot = 'SBER'
date_to_plot = 20190603
time_to_plot = 115023000000
buylevels=20
selllevels=20
color_black = '#000000'
color_light_black = '#C0C0C0'

lob = LOB(ticker=ticker_to_plot, date=date_to_plot, time=time_to_plot)

table_lob = lob.table_build()

table_lob.head()

buy = (table_lob.loc[table_lob['BUYSELL'] == 'B', ['PRICE', 'VOLUME']]).reset_index(drop=True)
sell = (table_lob.loc[table_lob['BUYSELL'] == 'S', ['PRICE', 'VOLUME']]).reset_index(drop=True)

# code to make beautiful barplot
bid = np.max(buy.PRICE)
ask = np.min(sell.PRICE)

figure, ax = plt.subplots(1, 1, sharey=True)
ax.barh(sell.PRICE[-selllevels:].reset_index(drop=True), 
        -sell.VOLUME[-selllevels:].reset_index(drop=True), 
        height=0.01, label = 'SELL', color = color_black)
ax.barh(buy.PRICE[:buylevels].reset_index(drop=True), 
         buy.VOLUME[:buylevels].reset_index(drop=True),
         height=0.01, label = 'BUY', color = color_light_black)

ax.ticklabel_format(axis='x', style='sci', scilimits=(0, 0))
ax.title.set_text('LOB for ' + ticker_to_plot +\
          ' at ' + str(date_to_plot)[:4] + '-' + str(date_to_plot)[4:6] + '-' + str(date_to_plot)[6:] +\
          ': ' +\
          str(time_to_plot)[:2] + ':' + str(time_to_plot)[2:4] + ':' + str(time_to_plot)[4:6] + '.' + str(time_to_plot)[6:])
ax.set_ylabel('PRICE')
ax.set_xlabel('VOLUME')

plt.legend(['ASK = ' + str(ask), 
            'BID = ' + str(bid)])
plt.grid()

figure.set_size_inches(w=5, h=3)
plt.savefig(os.path.join(PATH, 'figures_to_report/limit_order_book.pgf'), bbox_inches='tight')

# 3.  Эмпирическая функция трансакционных издержек

In [15]:
class Perold(LOB):
    '''This class implements the logic for Perold transaction costs calculation
    '''
    def __init__(self, ticker, date, time):
        '''
        :ticker: the string of fin. instrument's name ('SBER', for example)
        :date: the date in int (or str) format (for example, 20190603)
        :time: the time in int format (for example, 111050000000)
        '''
        super().__init__(ticker)
        self._date = date
        self._time = time
        self._lob = self.table_build(date, time)
        self._bid = np.max(self._lob.loc[self._lob['BUYSELL'] == 'B', 'PRICE'])
        self._ask = np.min(self._lob.loc[self._lob['BUYSELL'] == 'S', 'PRICE'])
        self._halfbidask = (self._bid + self._ask) / 2
        
        self._tr_costs = None
    
    
    
    def empirical_cost_function_build(self):
        '''This method builds the pd.DataFrame object that consists the following fields:
        VOLUME, COSTS according to the Perold's function θ(·)= Σ(p_i - p)V_i
        '''
        lob = self._lob
        lob['VOLUME'] = lob['VOLUME'].astype(int)
        buy = lob.loc[lob['BUYSELL'] == 'B', ['PRICE', 'VOLUME']].sort_values(by='PRICE', ascending=False).reset_index(drop=True)
        
        tr_costs_sell = copy.deepcopy(buy)
        tr_costs_sell['COSTS'] = np.cumsum(np.abs(tr_costs_sell['PRICE'] - self._halfbidask) * tr_costs_sell['VOLUME'])
        tr_costs_sell['VOLUME'] = np.cumsum(tr_costs_sell['VOLUME'])
        tr_costs_sell = tr_costs_sell[['VOLUME', 'COSTS']]
        #tr_costs_sell['VOLUME'] = -tr_costs_sell['VOLUME']
        
        
        sell = lob.loc[lob['BUYSELL'] == 'S', ['PRICE', 'VOLUME']].sort_values(by='PRICE', ascending=True).reset_index(drop=True)

        tr_costs_buy = copy.deepcopy(sell)
        tr_costs_buy['COSTS'] = np.cumsum(np.abs(tr_costs_buy['PRICE'] - self._halfbidask) * tr_costs_buy['VOLUME'])
        tr_costs_buy['VOLUME'] = np.cumsum(tr_costs_buy['VOLUME'])
        tr_costs_buy = tr_costs_buy[['VOLUME', 'COSTS']]
        tr_costs_buy['VOLUME'] = -tr_costs_buy['VOLUME']
        
        tr_costs_all = pd.concat((tr_costs_buy, tr_costs_sell)).sort_values(by='VOLUME').reset_index(drop=True)
        
        
        self._tr_costs = tr_costs_all

        return self._tr_costs

    def plot_empirical_function(self, path=None):
        '''This method plots the empirical function of costs and saves it into the given path (if None, 
        it will not be saved)
        :path: the string value (relative path where to save the plot), for example (./graph.pdf)
        '''
        if self._tr_costs is None:
            print('Just build the cost table at once')
        else:
            plt.figure()
            plt.plot(self._tr_costs['VOLUME'], self._tr_costs['COSTS'])
            plt.xticks(rotation=15)
            plt.title('Empirical Function of Transaction Costs')
            plt.xlabel('VOLUME, positive for sell, negative for buy')
            plt.ylabel('TRANSACTION COSTS')
            if path is None:
                plt.show() 
            else:
                plt.savefig(os.path.join(os.path.abspath(''), path), bbox_inches='tight')
                plt.close()

In [16]:
ticker_to_plot = 'SBER'
date_to_plot = 20190603
time_to_plot = 115023000000
buylevels=20
selllevels=20
color_black = '#000000'
color_light_black = '#C0C0C0'

perold = Perold(ticker=ticker_to_plot, date=date_to_plot, time=time_to_plot)

In [17]:
dataframe = perold.empirical_cost_function_build()

costs = dataframe.COSTS
volume = dataframe.VOLUME

figure, ax = plt.subplots(1, 1, sharey=True)

ax.plot(volume, costs, color=color_black)
ax.ticklabel_format(axis='x', style='sci', scilimits=(0, 0))
ax.ticklabel_format(axis='y', style='sci', scilimits=(0, 0))
ax.title.set_text('Empirical Function of Transaction Costs')
ax.set_ylabel('Transaction Costs')
ax.set_xlabel('VOLUME')
ax.grid(color='#DCDCDC')
figure.set_size_inches(w=5, h=3)
plt.savefig(os.path.join(PATH, 'figures_to_report/transaction_costs.pgf'), bbox_inches='tight')

# 4. Динамика сжатости и глубины согласно логарифмической модели

In [18]:
ticker = 'SBER'
path_to_file = os.path.join(PATH, 
                            'additional_data_for_figures_tables/'+\
                            'liquidity_time_series_aggregated/'+\
                            'SBER_liquidity_parameters.csv')
data = pd.read_csv(path_to_file, index_col=0)
data = data.loc[data['SECCODE'] != 'SBERP'].reset_index(drop=True)
data.DATETIME = list(map(lambda dt: datetime.datetime.strptime(dt, '%Y-%m-%d %H:%M:%S'), data.DATETIME))

In [19]:
black = '#000000'
light_black = '#C0C0C0'

figure, (ax1, ax2) = plt.subplots(1, 2, sharey=False)

ax1.plot(data.DATETIME, data.Alpha_B, color=light_black)
ax1.plot(data.DATETIME, data.Alpha_S, color=black)
# ax1.ticklabel_format(axis='y', style='sci', scilimits=(0, 0))
ax1.set_ylabel('Alpha')
ax1.title.set_text('Tightness for SBER')
ax1.legend(['Buy', 'Sell'])

ax2.plot(data.DATETIME, data.Beta_B, color=light_black)
ax2.plot(data.DATETIME, data.Beta_S, color=black)
# ax2.ticklabel_format(axis='y', style='sci', scilimits=(0, 0))
ax2.set_ylabel('Beta')
ax2.title.set_text('Depth for SBER')
ax2.legend(['Buy', 'Sell'])

figure.autofmt_xdate()
figure.set_size_inches(w=6, h=4)
plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)
plt.savefig(os.path.join(PATH, 'figures_to_report/tightness_depth.pgf'), bbox_inches='tight')

# 5. Полиномиальная параметризация модели 

In [20]:
ticker_to_plot = 'SBER'
date_to_plot = 20190603
time_to_plot = 115023000000
color_black = '#000000'
color_light_black = '#C0C0C0'

perold = Perold(ticker=ticker_to_plot, date=date_to_plot, time=time_to_plot)
transaction_costs_df = perold.empirical_cost_function_build()

volume = transaction_costs_df.VOLUME
costs = transaction_costs_df.COSTS

In [21]:
y = costs
X = np.stack([volume, volume**2, volume**3], axis=1)
            
model_full = linear_model.OLS(y, X)
results_full = model_full.fit()
y_full = results_full.predict(X)

mse_full = "{:.2e}".format(mse(y, y_full))

In [22]:
y = costs
X = np.stack([volume**2, volume**3], axis=1)
            
model_trunc = linear_model.OLS(y, X)
results_trunc = model_trunc.fit()
y_trunc = results_trunc.predict(X)

mse_trunc = "{:.2e}".format(mse(y, y_trunc))

In [23]:
y = costs
X = np.stack([volume, volume**2], axis=1)
            
model_squared = linear_model.OLS(y, X)
results_squared = model_squared.fit()
y_squared = results_squared.predict(X)

mse_squared = "{:.2e}".format(mse(y, y_squared))

In [24]:
figure, (ax1, ax2, ax3) = plt.subplots(3, 1, sharey=True)

ax1.plot(volume, y, color='#C0C0C0')
ax2.plot(volume, y, color='#C0C0C0')
ax3.plot(volume, y, color='#C0C0C0')

ax1.plot(volume, y_full, color='#000000')
ax2.plot(volume, y_trunc, color='#000000')
ax3.plot(volume, y_squared, color='#000000')

ax3.set_xlabel('VOLUME')
ax1.ticklabel_format(axis='x', style='sci', scilimits=(0, 0))
ax2.ticklabel_format(axis='x', style='sci', scilimits=(0, 0))
ax3.ticklabel_format(axis='x', style='sci', scilimits=(0, 0))
ax1.set_ylabel('Transaction Costs')
ax2.set_ylabel('Transaction Costs')
ax3.set_ylabel('Transaction Costs')
ax1.legend(['real',
           r'$\beta_1v_t + \beta_2v_t^2 + \beta_3v_t^3$' + '\nMSE = ' + mse_full])
ax2.legend(['real',
            r'$\beta_1v_t^2 + \beta_2v_t^3$' + '\nMSE = ' + mse_trunc]) 
ax3.legend(['real',
            r'$\beta_1v_t + \beta_2v_t^2$' + '\nMSE = ' + mse_squared])
ax1.grid(color='#DCDCDC')
ax2.grid(color='#DCDCDC')
ax3.grid(color='#DCDCDC')

figure.set_size_inches(w=5, h=7)

plt.tight_layout(pad=0.4)
plt.savefig(os.path.join(PATH, 'figures_to_report/parameterization_costs.pgf'), bbox_inches='tight')

# 6. Проверка результатов моделирования 

In [25]:
data_final = pd.read_csv(os.path.join(PATH, 'results_from_check/data_diff.csv'), index_col=0)
data_final['datetime'] = list(map(lambda x: time_convertion(x[1], date=x[0]), 
                                  zip(data_final['date'], data_final['time'])))
data_final

,date,equal,first,optimal,time,datetime
0,20190624,2.446319e+08,2.436011e+08,2.446321e+08,110000000000,2019-06-24 11:00:00
1,20190624,2.446317e+08,2.436405e+08,2.446311e+08,110500000000,2019-06-24 11:05:00
2,20190624,2.446842e+08,2.436754e+08,2.446837e+08,111000000000,2019-06-24 11:10:00
3,20190624,2.446809e+08,2.436772e+08,2.446814e+08,111500000000,2019-06-24 11:15:00
4,20190624,2.446211e+08,2.436956e+08,2.446216e+08,112000000000,2019-06-24 11:20:00
5,20190624,2.447814e+08,2.435792e+08,2.447783e+08,112500000000,2019-06-24 11:25:00
6,20190624,2.450683e+08,2.437356e+08,2.450662e+08,113000000000,2019-06-24 11:30:00
7,20190624,2.451881e+08,2.438638e+08,2.451875e+08,113500000000,2019-06-24 11:35:00
8,20190624,2.450969e+08,2.443260e+08,2.450992e+08,114000000000,2019-06-24 11:40:00
9,20190624,2.449943e+08,2.442702e+08,2.449947e+08,114500000000,2019-06-24 11:45:00


In [26]:
earlier = data_final.loc[data_final['date'] == 20190603].reset_index(drop=True)
later = data_final.loc[data_final['date'] == 20190624].reset_index(drop=True)

print('Naive strategy is the best for earlier date at {}% cases'.format(
np.round(100 * np.sum((earlier['equal'] - earlier['optimal']) > 0) / len(earlier['equal']), 2)
))

print('Optimal strategy is the best for later date at {}% cases'.format(
np.round(100 * np.sum((later['optimal'] - later['equal']) > 0) / len(later['equal']), 2)
))

Naive strategy is the best for earlier date at 72.0% cases
Optimal strategy is the best for later date at 52.0% cases


In [27]:
figure, (ax1, ax2) = plt.subplots(1, 2, sharey=False)

ax1.plot(earlier['datetime'], earlier['optimal'] - earlier['equal'], color='#000000')
ax2.plot(later['datetime'], later['optimal'] - later['equal'], color='#000000')

ax1.ticklabel_format(axis='y', style='sci', scilimits=(0, 0))
ax2.ticklabel_format(axis='y', style='sci', scilimits=(0, 0))

ax1.set_xlabel('Datetime, dd HH:MM')
ax2.set_xlabel('Datetime, dd HH:MM')

ax1.set_ylabel('Value, rub')

ax1.grid(color='#DCDCDC')
ax2.grid(color='#DCDCDC')

figure.autofmt_xdate()

figure.set_size_inches(w=6, h=4)
plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)
plt.savefig(os.path.join(PATH, 'figures_to_report/comparison_result.pgf'), bbox_inches='tight')

In [28]:
figure, (ax1, ax2) = plt.subplots(1, 2, sharey=False)

ax1.plot(earlier['datetime'], earlier['optimal'], color='#000000')
ax1.plot(earlier['datetime'], earlier['first'], color='#C0C0C0')
ax2.plot(later['datetime'], later['optimal'], color='#000000')
ax2.plot(later['datetime'], later['first'], color='#C0C0C0')

ax1.ticklabel_format(axis='y', style='sci', scilimits=(0, 0))
ax2.ticklabel_format(axis='y', style='sci', scilimits=(0, 0))

ax1.set_xlabel('Datetime, dd HH:MM')
ax2.set_xlabel('Datetime, dd HH:MM')

ax1.set_ylabel('Value, rub')

ax1.legend(['Optimal Strategy', 'All-at-First Strategy'])

ax1.grid(color='#DCDCDC')
ax2.grid(color='#DCDCDC')

figure.autofmt_xdate()

figure.set_size_inches(w=6, h=4)
plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)
plt.savefig(os.path.join(PATH, 'figures_to_report/liq_value_result.pgf'), bbox_inches='tight')